# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Nov 8 2022 11:35AM,250302,10,0086150940,ISDIN Corporation,Released
1,Nov 8 2022 11:35AM,250302,10,0086150941,ISDIN Corporation,Released
2,Nov 8 2022 11:35AM,250302,10,0086150949,ISDIN Corporation,Released
3,Nov 8 2022 11:35AM,250302,10,0086150956,ISDIN Corporation,Released
4,Nov 8 2022 11:35AM,250302,10,0086150957,ISDIN Corporation,Released
5,Nov 8 2022 11:35AM,250302,10,0086151028,ISDIN Corporation,Released
6,Nov 8 2022 11:35AM,250302,10,0086151034,ISDIN Corporation,Released
7,Nov 8 2022 11:35AM,250302,10,0086151123,ISDIN Corporation,Released
8,Nov 8 2022 11:35AM,250302,10,0086151153,ISDIN Corporation,Released
9,Nov 8 2022 11:35AM,250302,10,0086151152,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
35,250298,Released,2
36,250299,Released,1
37,250300,Released,1
38,250301,Released,1
39,250302,Released,14


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250298,NaN,NaN,NaN,2.0
250299,NaN,NaN,NaN,1.0
250300,NaN,NaN,NaN,1.0
250301,NaN,NaN,NaN,1.0
250302,NaN,NaN,NaN,14.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250136,0.0,0.0,0.0,24.0
250194,0.0,0.0,86.0,19.0
250204,0.0,0.0,0.0,1.0
250206,0.0,0.0,0.0,1.0
250235,0.0,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Cancelled,Completed,Executing,Released
id,,,,
250136,0.0,0,0,24
250194,0.0,0,86,19
250204,0.0,0,0,1
250206,0.0,0,0,1
250235,0.0,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250136,0.0,0,0,24
1,250194,0.0,0,86,19
2,250204,0.0,0,0,1
3,250206,0.0,0,0,1
4,250235,0.0,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Completed,Executing,Released
0,250136,0.0,,,24
1,250194,0.0,,86,19
2,250204,0.0,,,1
3,250206,0.0,,,1
4,250235,0.0,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Completed     object
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation
14,Nov 8 2022 11:30AM,250301,10,Methapharm-G
15,Nov 8 2022 11:29AM,250300,10,Methapharm-G
16,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC"
17,Nov 8 2022 11:20AM,250298,10,Emerginnova
19,Nov 8 2022 11:18AM,250297,10,Emerginnova
20,Nov 8 2022 11:14AM,250296,10,"Citieffe, Inc."
22,Nov 8 2022 11:14AM,250295,19,"AdvaGen Pharma, Ltd"
28,Nov 8 2022 11:01AM,250291,19,"Methapharm, Inc."
29,Nov 8 2022 10:56AM,250290,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Completed,Executing,Released
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation,0.0,,,14
1,Nov 8 2022 11:30AM,250301,10,Methapharm-G,0.0,,,1
2,Nov 8 2022 11:29AM,250300,10,Methapharm-G,0.0,,,1
3,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC",0.0,,,1
4,Nov 8 2022 11:20AM,250298,10,Emerginnova,0.0,,,2
5,Nov 8 2022 11:18AM,250297,10,Emerginnova,0.0,,,1
6,Nov 8 2022 11:14AM,250296,10,"Citieffe, Inc.",0.0,,,2
7,Nov 8 2022 11:14AM,250295,19,"AdvaGen Pharma, Ltd",0.0,,,6
8,Nov 8 2022 11:01AM,250291,19,"Methapharm, Inc.",0.0,,,1
9,Nov 8 2022 10:56AM,250290,20,"Exact-Rx, Inc.",0.0,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation,14,,
1,Nov 8 2022 11:30AM,250301,10,Methapharm-G,1,,
2,Nov 8 2022 11:29AM,250300,10,Methapharm-G,1,,
3,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC",1,,
4,Nov 8 2022 11:20AM,250298,10,Emerginnova,2,,
5,Nov 8 2022 11:18AM,250297,10,Emerginnova,1,,
6,Nov 8 2022 11:14AM,250296,10,"Citieffe, Inc.",2,,
7,Nov 8 2022 11:14AM,250295,19,"AdvaGen Pharma, Ltd",6,,
8,Nov 8 2022 11:01AM,250291,19,"Methapharm, Inc.",1,,
9,Nov 8 2022 10:56AM,250290,20,"Exact-Rx, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation,14,,
1,Nov 8 2022 11:30AM,250301,10,Methapharm-G,1,,
2,Nov 8 2022 11:29AM,250300,10,Methapharm-G,1,,
3,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC",1,,
4,Nov 8 2022 11:20AM,250298,10,Emerginnova,2,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation,14.0,NaN,NaN
1,Nov 8 2022 11:30AM,250301,10,Methapharm-G,1.0,NaN,NaN
2,Nov 8 2022 11:29AM,250300,10,Methapharm-G,1.0,NaN,NaN
3,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC",1.0,NaN,NaN
4,Nov 8 2022 11:20AM,250298,10,Emerginnova,2.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Nov 8 2022 11:35AM,250302,10,ISDIN Corporation,14.0,0.0,0.0
1,Nov 8 2022 11:30AM,250301,10,Methapharm-G,1.0,0.0,0.0
2,Nov 8 2022 11:29AM,250300,10,Methapharm-G,1.0,0.0,0.0
3,Nov 8 2022 11:27AM,250299,19,"Quagen Pharmaceuticals, LLC",1.0,0.0,0.0
4,Nov 8 2022 11:20AM,250298,10,Emerginnova,2.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3503902,31.0,17.0,0.0
12,250262,1.0,0.0,0.0
15,500330,43.0,86.0,0.0
16,250244,0.0,1.0,0.0
19,1001152,9.0,0.0,0.0
20,2252530,16.0,7.0,15.0
21,1000897,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3503902,31.0,17.0,0.0
1,12,250262,1.0,0.0,0.0
2,15,500330,43.0,86.0,0.0
3,16,250244,0.0,1.0,0.0
4,19,1001152,9.0,0.0,0.0
5,20,2252530,16.0,7.0,15.0
6,21,1000897,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,31.0,17.0,0.0
1,12,1.0,0.0,0.0
2,15,43.0,86.0,0.0
3,16,0.0,1.0,0.0
4,19,9.0,0.0,0.0
5,20,16.0,7.0,15.0
6,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,31.0
1,12,Released,1.0
2,15,Released,43.0
3,16,Released,0.0
4,19,Released,9.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
Executing,17.0,0.0,86.0,1.0,0.0,7.0,0.0
Released,31.0,1.0,43.0,0.0,9.0,16.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
1,Executing,17.0,0.0,86.0,1.0,0.0,7.0,0.0
2,Released,31.0,1.0,43.0,0.0,9.0,16.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,0.0,0.0,0.0,0.0,0.0,15.0,0.0
1,Executing,17.0,0.0,86.0,1.0,0.0,7.0,0.0
2,Released,31.0,1.0,43.0,0.0,9.0,16.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()